# 決定木
分類と回帰の両方を実行できる柔軟性の高いアルゴリズムで複雑なデータセットに適合できる。  
決定木は、機械学習のアルゴリズムの中でも有数の力をもつランダムフォレストの基本構成要素である。

## 決定木の訓練と可視化
とりあえず決定木を作成して予測を行う。

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier